Stage 2:  Data preprocessing stage to produce structured data in csv format also stored in Cloud Object Storage. As columns in the csv file we suggest date, geographic location, url, and sentiment analysis.

In [1]:
import lithops
from transformers import pipeline

In [2]:
def analyze(string_list):
  # this model is the best one, I tested the others available and they're quite bad...
  classifier = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")
  results = classifier(string_list)
  return results

In [8]:
to_analyze = [
              ["We hope you don't hate it.", "Hello how are you?", "You're beautiful", "Nice work there"],
              ["Likewise, if the Spanish government had invested more resources in researching a vaccine, it would not have to be asking for them, it could even be selling to other countries.  You have to think a little more", "te he visto en plaza virreina! Vaya pedo has pillado, y sin mascarilla"]
              ]

# Check if types are valid 
assert type(to_analyze) is list
for supposed_list in to_analyze:
  assert type(supposed_list) is list

In [9]:
fexec = lithops.FunctionExecutor(runtime='gilarasa/lithops-cloudbutton-challenge-py3.9:0.2', runtime_memory=2048)
all_invocations = fexec.map(analyze, to_analyze)
  
for result in fexec.get_result(all_invocations):
  for inferences in result:
    print(inferences)

2021-05-10 21:08:42,516 [INFO] lithops.config -- Lithops v2.3.3
2021-05-10 21:08:42,528 [INFO] lithops.storage.backends.ibm_cos.ibm_cos -- IBM COS Storage client created - Region: eu-de
2021-05-10 21:08:44,472 [INFO] lithops.serverless.backends.ibm_cf.ibm_cf -- IBM CF client created - Region: eu-de - Namespace: Namespace-n4e-colab
2021-05-10 21:08:44,474 [INFO] lithops.executors -- Serverless Executor created with ID: 10d114-3
2021-05-10 21:08:44,476 [INFO] lithops.invokers -- ExecutorID 10d114-3 | JobID M000 - Selected Runtime: gilarasa/lithops-cloudbutton-challenge-py3.9:0.1 - 2048MB
2021-05-10 21:08:44,490 [INFO] lithops.job.job -- ExecutorID 10d114-3 | JobID M000 - Uploading function and data - Total: 1.1KiB
2021-05-10 21:08:45,083 [INFO] lithops.invokers -- ExecutorID 10d114-3 | JobID M000 - Starting function invocation: analyze() - Total: 2 activations
2021-05-10 21:08:45,089 [INFO] lithops.invokers -- ExecutorID 10d114-3 | JobID M000 - View execution logs at /tmp/lithops/logs/10